In [1]:
import argparse
import os
import sys
import pandas as pd
import optuna
from optuna.integration import PyTorchLightningPruningCallback

import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from glob import glob 

from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.profilers import PyTorchProfiler

sys.path.append(os.pardir)
from src.config import *
from src.dataset import DataModule
from src.model import LightningNetwork

In [ ]:
base_model_name = 'efficientnet_b0'
model = LightningNetwork(
        base_model=get_base_model(base_model_name),
        dropout=0.2, 
        output_dims=[128, 64],
        learning_rate=0.002
)
data_module = DataModule(model_name=base_model_name)

: 

In [ ]:
logger = TensorBoardLogger(
    save_dir="../logs/tb_logs", 
    name="trail_v1"
)
profiler = PyTorchProfiler(
    on_trace_ready=torch.profiler.tensorboard_trace_handler("../logs/tb_logs/profiler0"),
    schedule=torch.profiler.schedule(skip_first=10, wait=1, warmup=1, active=20),
)


trainer = pl.Trainer(
    profiler=profiler,
    logger=logger,
    # accelerator=ACCELERATOR,
    # devices=DEVICES,
    min_epochs=1,
    max_steps=2,
    max_epochs=NUM_EPOCHS,
    # precision='bf16-mixed',
    precision=32,
    fast_dev_run=True
    # callbacks=[EarlyStopping(monitor="val_loss")],
)
# trainer.fit(model, data_module)
trainer.validate(model, data_module)
# trainer.test(model, data_module)

: 

: 

In [ ]:
import torch
import pytorch_lightning as pl
from model import NN
from dataset import MnistDataModule
import config
#TODO: import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.profilers import PyTorchProfiler

torch.set_float32_matmul_precision("medium") # to make lightning happy

if __name__ == "__main__":
    logger = TensorBoardLogger("tb_logs", name="mnist_model_v1")
    profiler = PyTorchProfiler(
        on_trace_ready=torch.profiler.tensorboard_trace_handler("tb_logs/profiler0"),
        schedule=torch.profiler.schedule(skip_first=10, wait=1, warmup=1, active=20),
    )
    model = NN(
        input_size=config.INPUT_SIZE,
        learning_rate=config.LEARNING_RATE,
        num_classes=config.NUM_CLASSES,
    )
    dm = MnistDataModule(
        data_dir=config.DATA_DIR,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
    )
    trainer = pl.Trainer(
        profiler=profiler,
        logger=logger,
        accelerator=config.ACCELERATOR,
        devices=config.DEVICES,
        min_epochs=1,
        max_epochs=config.NUM_EPOCHS,
        precision=config.PRECISION,
        # callbacks=[EarlyStopping(monitor="val_loss")],
    )
    trainer.fit(model, dm)
    trainer.validate(model, dm)
    trainer.test(model, dm)

In [2]:
img_paths = glob(f'{DATA_DIR}/images/*')
df = pd.read_csv(f'{DATA_DIR}/verified_annotation_from_xml.csv')

df = pd.read_csv(f'{DATA_DIR}/verified_annotation_from_xml.csv')
df['img_path'] =f'{DATA_DIR}/images/' + df['image_name']
df.drop(columns=['Unnamed: 0'], inplace=True)
df['label_name'] = df['label_name'].apply(lambda x: x.lower())
df['sparse_label'] = df['label_name'].map({'atopic': 0, 'papular': 1,'scabies': 2})
df.head()


,label_name,xmin,ymin,xmax,ymax,image_name,image_width,image_height,patient_id,img_path,sparse_label
0,papular,0,919,1635,2918,00241a_A_P.jpg,1960,4032,3.0,D:/Projects/Skin Disease Detection/Dataset/Lat...,1
1,scabies,1006,1260,1713,2143,00240b_A_S.jpg,1960,4032,6.0,D:/Projects/Skin Disease Detection/Dataset/Lat...,2
2,atopic,215,424,457,706,0052b_A_A.jpg,720,1280,9.0,D:/Projects/Skin Disease Detection/Dataset/Lat...,0
3,scabies,138,94,633,643,0044a_A_S.jpg,1280,1280,14.0,D:/Projects/Skin Disease Detection/Dataset/Lat...,2
4,scabies,354,866,950,1217,0044a_A_S.jpg,1280,1280,14.0,D:/Projects/Skin Disease Detection/Dataset/Lat...,2


In [4]:
import torch
from torch import nn
from typing import List
import torch.nn.functional as F
from torch import nn, optim
import pytorch_lightning as pl
import torchmetrics

import os
import sys
sys.path.append(os.pardir)
from src.config import *


In [ ]:
LightningNetwork(
    base_model = efficientnet_b0(),
    dropout=0.3, 
    output_dims=[128, 64])

In [ ]:
import torch
import pytorch_lightning as pl
from model import NN
from dataset import MnistDataModule
import config
#TODO: import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.profilers import PyTorchProfiler

torch.set_float32_matmul_precision("medium") # to make lightning happy

if __name__ == "__main__":
    logger = TensorBoardLogger("tb_logs", name="mnist_model_v1")
    profiler = PyTorchProfiler(
        on_trace_ready=torch.profiler.tensorboard_trace_handler("tb_logs/profiler0"),
        schedule=torch.profiler.schedule(skip_first=10, wait=1, warmup=1, active=20),
    )
    model = NN(
        input_size=config.INPUT_SIZE,
        learning_rate=config.LEARNING_RATE,
        num_classes=config.NUM_CLASSES,
    )
    dm = MnistDataModule(
        data_dir=config.DATA_DIR,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
    )
    trainer = pl.Trainer(
        profiler=profiler,
        logger=logger,
        accelerator=config.ACCELERATOR,
        devices=config.DEVICES,
        min_epochs=1,
        max_epochs=config.NUM_EPOCHS,
        precision=config.PRECISION,
        # callbacks=[EarlyStopping(monitor="val_loss")],
    )
    trainer.fit(model, dm)
    trainer.validate(model, dm)
    trainer.test(model, dm)